In [3]:
class MyClass:
    def __init__(self):
        self.id = 1234
        self.name = "Alex"
        self._smthng = "woa"

obj = MyClass()

fields = [attr for attr in dir(obj) if not attr.startswith('_')]

for field in fields:
    print(field)


id
name


In [13]:
class MyClass:
    def __init__(self):
        self.id = 1234
        self.name = "Alex"
        self._smthng = "woa"
        
    def get_name(self):
        return self.name
    
obj = MyClass()

print(obj.get_name())

my_get_name = getattr(obj, "get_name")
print(my_get_name())

Alex
Alex


In [14]:
class A:
    pass

class B(A):
    pass

class C(B):
    pass


In [16]:
def get_inheritance(cls):
    return ' -> '.join([c.__name__ for c in cls.__mro__])
print(get_inheritance(OSError))

OSError -> Exception -> BaseException -> object


In [2]:
class SVG:
    def __init__(self):
        self.lines = []
        self.circles = []

    def line(self, x1, y1, x2, y2, color='black'):
        self.lines.append((x1, y1, x2, y2, color))

    def circle(self, cx, cy, r, color='black'):
        self.circles.append((cx, cy, r, color))

    def save(self, filename, width, height):
        svg_file = open(filename, 'w')
        svg_file.write(f'<svg version="1.1" width="{width:.6f}" height="{height:.6f}" xmlns="http://www.w3.org/2000/svg">\n')
        for x1, y1, x2, y2, color in self.lines:
            svg_file.write(f'<line x1="{x1:.6f}" y1="{y1:.6f}" x2="{x2:.6f}" y2="{y2:.6f}" stroke="{color}" />\n')
        for cx, cy, r, color in self.circles:
            svg_file.write(f'<circle cx="{cx:.6f}" cy="{cy:.6f}" r="{r:.6f}" fill="{color}" />\n')
        svg_file.write('</svg>')
        svg_file.close()

svg = SVG()

svg.line(10, 10, 60, 10, color='black')
svg.line(60, 10, 60, 60, color='black')
svg.line(60, 60, 10, 60, color='black')
svg.line(10, 60, 10, 10, color='black')

svg.circle(10, 10, r=5, color='red')
svg.circle(60, 10, r=5, color='green')
svg.circle(60, 60, r=5, color='red')
svg.circle(10, 60, r=5, color='blue')

svg.save('pic.svg', 100, 100)

In [21]:
class Num:
    def __init__(self, value):
        self.value = value
    
    def get_content(self):
        return self.value
    
class Add:
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
    def get_content(self):
        return "+"
    
    
class Mul:
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
    def get_content(self):
        return "*"

In [23]:
class PrintVisitor:
    def visit_Num(self, node):
        return node.get_content()
    
    def visit_Add(self, node):
        return self.visit_Operation(node)
    
    def visit_Mul(self, node):
        return self.visit_Operation(node)
    
    def visit_Operation(self, node):
        return f"({self.visit(node.left)} "+ \
            f"{node.get_content()} " + \
            f"{self.visit(node.right)})"
    
    def visit(self, node):
        method = f"visit_{type(node).__name__}"
        method = getattr(self, method)
        return(method(node))
    
    
ast = Add(Num(7), Mul(Num(3), Num(2)))    
pv = PrintVisitor()
print(pv.visit(ast))

(7 + (3 * 2))


In [24]:
class CalcVisitor:
    def visit_Num(self, node):
        return node.get_content()
    
    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right)
    
    def visit_Mul(self, node):
        return self.visit(node.left) * self.visit(node.right)
    
    def visit(self, node):
        method = f"visit_{type(node).__name__}"
        method = getattr(self, method)
        return(method(node))
    
    
ast = Add(Num(7), Mul(Num(3), Num(2)))    
cv = CalcVisitor()
print(cv.visit(ast))

13


In [30]:
class StackVisitor:
    def visit_Num(self, node):
        return f"PUSH {node.get_content()}\n"
    
    def visit_Add(self, node):
        return f"{self.visit(node.left)}"+ \
            f"{self.visit(node.right)}ADD\n"
    
    def visit_Mul(self, node):
        return f"{self.visit(node.left)}"+ \
            f"{self.visit(node.right)}MUL\n"
    
    def visit(self, node):
        method = f"visit_{type(node).__name__}"
        method = getattr(self, method)
        return(method(node))
    
    
ast = Add(Num(7), Mul(Num(3), Num(2)))    
sv = StackVisitor()
print(sv.visit(ast))

PUSH 7
PUSH 3
PUSH 2
MUL
ADD



In [17]:
#5.2
scale_x = 25
scale_y = 50

class Tree:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        self.x = 0
        self.y = 0
    
    def getWH(self):
        (w,h) = (4+scale_x, 4+scale_x)
        if self.left:
            (w1,h1) = self.left.getWH()
            (w,h) = (w+w1, h+h1)
        if self.right:
            (w1,h1) = self.right.getWH()
            (w,h) = (w+w1, h+h1)
        return (w,h)
            

    def layout(self, x=50, y=2, level=1):
        self.x = x
        self.y = y
        if self.left:
            self.left.layout(x - scale_x/level, y + scale_y, level+1)
        if self.right:
            self.right.layout(x + scale_x/level, y + scale_y, level+1)

    def draw(self, svg):
        if self.left:
            svg.line(self.x, self.y, self.left.x, self.left.y, color='black')
            self.left.draw(svg)
        if self.right:
            svg.line(self.x, self.y, self.right.x, self.right.y, color='black')
            self.right.draw(svg)
        svg.circle(self.x, self.y, r=4, color='black')

    def save(self, filename):
        self.layout()
        svg = SVG()
        self.draw(svg)
        (w,h) = self.getWH()
        svg.save(filename, w, h)

tree_2 = Tree(2, Tree(3, Tree(4), Tree(5)), Tree(6, Tree(7)))
tree_8 = Tree(8, Tree(9, Tree(10), Tree(11, Tree(12), Tree(13))), Tree(14))
tree = Tree(1, tree_2, tree_8)
tree.save('tree.svg')


In [26]:
#4.1
class Tag:
    def __init__(self, name):
        self.name = name
        self.attributes = []
        self.children = []

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        pass

    def __call__(self, *args, **kwargs):
        if args:
            text = ' '.join(str(arg) for arg in args)
            self.children.append(text)
        if kwargs:
            for key, value in kwargs.items():
                attr = f'{key}="{value}"'
                self.attributes.append(attr)

        return self

    def __str__(self):
        attrs = ' '.join(self.attributes)
        if attrs:
            attrs = f' {attrs}'
        children = ''.join(str(child) for child in self.children)
        return f'<{self.name}{attrs}>{children}</{self.name}>'


class HTML:
    def __init__(self):
        self.root = None
        self.current = None

    def __getattr__(self, name):
        def tag(*args, **kwargs):
            new_tag = Tag(name)
            if self.root is None:
                self.root = new_tag
            if self.current is not None:
                self.current.children.append(new_tag)
            new_tag(*args, **kwargs)
            self.current = new_tag
            return new_tag
        return tag

    def __enter__(self):
        return self.root

    def __exit__(self, type, value, traceback):
        pass

    def get_code(self):
        return str(self.root)



html = HTML()
with html:
    with html.body():
        with html.div():
            with html.div():
                html.p('Первая строка.')
                html.p('Вторая строка.')
            with html.div():
                html.p('Третья строка.')

print(html.get_code())


<body><div><div><p>Первая строка.<p>Вторая строка.<div><p>Третья строка.</p></div></p></p></div></div></body>
